In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelBinarizer
import os
import sys
module_path = os.path.abspath(os.path.join('/Users/briangraham/insight/trailrec/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import model

In [12]:
trail_data,indices = model.get_trail_data()
trail_data = model.get_clean_data(trail_data)

In [30]:
cosine_sim = model.model_tfidf(trail_data)
model.get_recommendations()

In [5]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(trail_data['description'])

In [170]:
trail_data.columns

Index(['index', 'trail_id', 'Altitude change', 'Altitude end', 'Altitude max',
       'Altitude min', 'Altitude start', 'Avg reverse time', 'Avg time',
       'Bike type', 'Difficulty rating', 'Direction', 'Distance climb',
       'Distance down', 'Global Ranking', 'Grade', 'Grade max', 'Grade min',
       'Physical rating', 'Ride in rain', 'Riding area', 'TTFs on trail',
       'Trail Usage', 'Trail type', 'avg_time', 'city', 'climb', 'descent',
       'description', 'distance', 'latitude', 'longitude', 'rating', 'state',
       'votes', 'Climb Difficulty', 'Distance flat'],
      dtype='object')

In [205]:
trail_data_categorical = trail_data[['Difficulty rating']]

In [206]:
trail_data_categorical = trail_data_categorical.apply(lambda x: x.str.replace(' ',''))
for col in trail_data_categorical.columns:
    trail_data_categorical[col] = trail_data_categorical[col].astype('category')
trail_data_categorical.dtypes

Difficulty rating    category
dtype: object

In [183]:
trail_data_numerical = trail_data[['Altitude change',
    'Altitude end', 'Altitude max','Altitude min',
    'Altitude start','Distance climb','Distance down',
    'Grade', 'Grade max', 'Grade min',
    'climb', 'descent','distance','rating','votes','Distance flat']]

In [21]:
trail_data_numerical.head()

,Altitude change,Altitude end,Altitude max,Altitude min,Altitude start,Distance climb,Distance down,Grade,Grade max,Grade min,climb,descent,distance,rating,votes,Distance flat
0,-1508.0,349,1857.0,349,1857,0.027462,2.000000,-18.592,-318.294,11.070,6,-1513,2.000000,4.55,13,NaN
1,-191.0,2446,2636.0,2446,2636,0.027462,0.379545,-8.579,-23.766,17.378,24,-215,0.421591,3.50,8,0.014394
2,-694.0,4931,5625.0,4918,5625,0.065152,0.472727,-24.429,-57.443,9.941,22,-716,0.537879,3.20,5,NaN
3,-46.0,1054,NaN,1054,1100,0.014583,0.244697,-3.242,-8.697,10.616,8,-54,0.268371,1.00,1,0.009091
4,-0.0,2638,2638.0,2635,2638,0.014015,0.021780,NaN,-4.765,3.158,3,-3,0.061364,4.00,1,0.012121


In [49]:
trail_data_numerical.isnull().sum()

Altitude end      0
Altitude min      0
Altitude start    0
Distance climb    2
Distance down     0
Grade max         0
climb             0
descent           0
distance          0
rating            0
votes             0
dtype: int64

In [216]:
trail_data_numerical = trail_data[['Altitude end','Altitude start',
                                   'Grade max','climb', 'descent','distance','rating']]

In [244]:
from sklearn import preprocessing
scaler_num = preprocessing.StandardScaler().fit(trail_data_numerical)
df_num_scaled = scaler_num.transform(trail_data_numerical)

In [245]:
enc = preprocessing.LabelBinarizer().fit(trail_data_categorical)
df_diff = enc.transform(trail_data_categorical)
scaler_cat = preprocessing.StandardScaler().fit(df_diff)
df_diff_scaled = scaler_cat.transform(df_diff)

/Users/briangraham/anaconda3/envs/trailrec2/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [255]:
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [260]:
all_features = np.concatenate([tfidf_matrix.toarray(),df_diff_scaled,df_num_scaled],axis=1)

In [261]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(all_features, all_features)

In [269]:
reclist,trail_in = model.get_recommendations(trail_data,'verboten',indices,cosine_sim)

In [2]:
reclist[['trail_id','description']]

NameError: name 'reclist' is not defined

In [66]:
# get difficulty
trail_data_cat = trail_data[['Difficulty rating']]
trail_data_cat = trail_data_cat.apply(lambda x: x.str.replace(' ','')).astype('category')

In [67]:
# get numerical data
trail_data_numerical = trail_data[['Altitude end','Altitude start',
    'Grade max','climb', 'descent','distance','rating']]

In [68]:
scaler_num = StandardScaler().fit(trail_data_numerical)
df_num_scaled = scaler_num.transform(trail_data_numerical)

In [69]:
enc = LabelBinarizer().fit(trail_data_cat)
df_cat = enc.transform(trail_data_cat)
scaler_cat = StandardScaler().fit(df_cat)
df_cat_scaled = scaler_cat.transform(df_cat)

/Users/briangraham/anaconda3/envs/trailrec2/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [70]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(trail_data['description']).toarray()
all_features = np.concatenate([tfidf_matrix,df_cat_scaled,df_num_scaled],axis=1)

In [63]:
cosine_sim = linear_kernel(all_features, all_features)
reclist,trail_in = model.get_recommendations(trail_data,'verboten',indices,cosine_sim)
reclist[['trail_id','description']]

,trail_id,description
1467,martha-creek-downhill,The trail drops from the tower into some techn...
1657,north-cinnabar,"This trail starts up on the ridge, is lost in ..."
1091,hopeful,Expect hiking traffic.Not part of the main Bik...
908,g-l-d,Connection from Goldie Lake to Hopeful. Use ...
2509,top-world,The Peak Zone and the new Top of the World alp...
404,castle-pass-trail,Trail has been reactivated through the efforts...
773,elbow-pass,River crossing can be problematic during perio...
116,atlas-shrugged,Did you earn your descent by pedalling to the ...
408,catapult-ranch,Trail has been built specifically for the Big ...
2259,south-caribou-pass,The trail to South Caribou Pass is complete.M...


In [64]:
cosine_sim = cosine_similarity(all_features, all_features)
reclist,trail_in = model.get_recommendations(trail_data,'verboten',indices,cosine_sim)
reclist[['trail_id','description']]

,trail_id,description
2216,slunt-sbomb,"This trail is probably the longest, flowiest, ..."
382,cabin-trail-wolf-s-den-wolfgangalang-19451,"steep, loamy, trail is only 1.5 months old. ro..."
2570,turducken,"Beginning at the end of High Roller, this dow..."
2589,ultimate-frisby-connector,Fast flowy trail with great views and fun chal...
1217,kashmir-6598,"Nearly 10 years old, this now classic technica..."
2743,will-power,This popular trail is a top to bottom single t...
2571,turnstyles,Bike park in the backcountry. The new big brow...
2495,today-s-special,A stunning descent from Castle Mountain. Start...
334,brokeback-ridge,It's pretty unlikely that you would find anyon...
1912,red-sonja,"Epic riding - fun, bermy trail back down to Bu..."
